# Analiza danych dotycząca samobójstw. 

## Wstęp

Według danych [National Vital Statistics System CDC](https://www.cdc.gov/nchs/products/databriefs/db330.htm) od 1999 do 2017 roku wskaźniki samobójstw wzrosły o 33%. Obserwując dane dotyczące obywatelów na przykład Stanów Zjednoczonych, nie sposób nie zauważyć wielu niepokojących faktów. Samobójstwa osiągają swój szczyt od 50 lat, oraz stały się obecnie drugą najczęstszą przyczyną śmierci dla osób w wieku poniżej 35 lat. Ta zwiększająca się liczba samobójstw może być wskaźnikiem narastających problemów społecznych.

Z pewnością istnieją jakieś czynniki, które mogą zwiększać ryzyko wystąpienia samobójstwa lub zmniejszać. W poniższej pracy przeanalizowane zostaną różne czynniki, które korelują ze wskaźnikami samobójstw w różnych krajach w latach 1985–2016. Najpierw przeanalizowane zostaną dane zawarte w wybranym zbiorze. Następnie poszukamy wszelkich korelacji ze wskaźnikami samobójstw wśród różnych czynników, takich jak wiek, płeć czy region geograficzny. 

### Wybrany zbiór danych

Używany w poniższej analizie zestaw danych to dane zebrane z czterech różnych źródeł i połączone według czasu i miejsca. Zbiór ten został utworzony, aby umożliwić znalezienie sygnałów skorelowanych ze wzrostem wskaźników samobójstw w całym spektrum społeczno-ekonomicznym na całym świecie. Zestaw ten nosi nazwę *Suicide Rates Overview 1985 to 2016*, a pobrany został z [kaggle](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016).

### Import potrzebnych bibliotek

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import json

### Przegląd zbioru

In [ ]:
# wczytanie danych z pliku csv i wykonanie kopii

data_original = pd.read_csv("data/suicides.csv")
df = data_original.copy()

Zobaczmy jak prezentują się dane. Poniżej wyświetlone zostanie 5 losowych rekordów ze zbioru oraz ogólne informacje o zbiorze.

In [ ]:
df.sample(5)

In [ ]:
# wymiar danych
df.shape

In [ ]:
# lista atrybutów zbioru 
list(df.columns)

Widać, że zbiór składa się z 12 column oraz 27820 rekordów. 

### Atrybuty zbioru

- country: nazwy 101 krajów, z których zebrano dane
- year: lata od 1985 do 2016
- sex: płeć
- age: przedziały wiekowe: '5-14' ,'15-24', '35-54', '75+', '25-34', '55-74'
- suicides_no: liczba samobójstw w danym roku, w danej grupie wiekowej, dla danej płci, w danym państwie
- population: ludność kraju w danym roku, w danej grupie wiekowej, dla danej płci, w danym państwie
- suicides/100k pop: liczba samobójstw na 100 tys. Mieszkańców, w danej grupie wiekowej, dla danej płci, w danym państwie
- country-year: kombinacja Kraj/Rok.
- HDI for year: wskaźnik rozwoju społecznego na dany rok
- gdp_for_year: PKB kraju na dany rok
- gdp_per_capita: PKB na mieszkańca dla kraju
- generation: pokolenie (na podstawie średniej grup wiekowych)

## Przygotowanie zbioru do analizy

Wstępne przetwarzanie danych jest jednym z najważniejszych kroków w procesie eksploracji danych, obejmującym przygotowanie i transformację początkowego zbioru danych. Chociaż sam zbiór jest stosunkowo czysty, nadal konieczne jest usunięcie wszelkich zbędnych danych, które nie są potrzebne do wykonania analizy.

### Typy danych 

Sprawdźmy więc najpierw czy typy danych są takie jakich oczekujemy.

In [ ]:
df.dtypes

Wygląda na to, że typy są prawidłowe oprócz kolumny *gdp_for_year*, która powinna mieć wartości numeryczne. Usunięte zostaną więc przecinki oraz zmieniony zostanie typ.

In [ ]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].str.replace(',','')
df[' gdp_for_year ($) '] = pd.to_numeric(df[' gdp_for_year ($) '])

### Brakujące wartości

In [ ]:
df.isnull().sum()

Jedna z kolumn zawiera wiele brakujących wartości. Sprawdźmy, jaką część zbioru stanowią brakujące dane.

In [ ]:
print('Brakujące dane w kolumnie HDI for year:', df['HDI for year'].isnull().sum() * 100 / len(df), '%')

Biorąc pod uwagę, że większość danych w kolumnie *HDI for year* posiada braki, możemy tą kolumne całkowicie usunąć gdyż i tak nie przyda się w dalszej analizie.

In [ ]:
df=df.drop(['HDI for year'],axis=1)

### Kolumny zbędne

W zbiorze znajdują się kolumny nie zawierające przydatnych dla nas informacji. Jedna z nich to *country-year*, która reprezentuje połączone dane z kolumn *year* oraz *country*. Mamy tutaj niepotrzebną redundancję, więc możemy się tych danych pozbyć. 

Drugą zbędną kolumną jest *generation*. Zawiera ona informację na temat pokoleń, których ramki czasowe określane są bardzo różnie. Ich definicja jest więc umowna. Poza tym informacja ta nie jest istotna, gdyż mamy w tabeli dane na temat wieku. Kolumna ta zostaje więc również usunięta.

In [ ]:
df=df.drop(['generation', 'country-year'],axis=1)

### Nazwy kolumn

Niektóre z kolumn można znacznie uprościć, aby pracowało się z nimi łatwiej.

In [ ]:
df.rename(index = str, columns = {'suicides/100k pop':'suiciderate'}, inplace=True)
df.rename(index = str, columns = {'gdp_per_capita ($)':'gdp'}, inplace=True)
df.rename(index = str, columns = {' gdp_for_year ($) ':'yearlygdp'}, inplace=True)

### Nazwy państw

Istnieje wiele bibliotek pozwalających na przedstawianie danych geograficznych na mapach. Aby skorzystać z takich wykresów należy posiadać nazwy państw zapisane w sposób, jaki jest akceptowalny przez daną bibliotekę lub w sposób jaki odpowiada danym zapisanym w pliku geo.json. W przypadku tej pracy nazwy państw zostaną zmienione tak, aby można było je wykorzystać z danymi geograficznymi zapisanymi w pliku. Plik ten został pobrany ze strony [https://geojson-maps.ash.ms/https://geojson-maps.ash.ms/](https://geojson-maps.ash.ms/).

In [ ]:
json_map_path = 'data/custom.geo.json'

with open(json_map_path) as f:
    geo_world = json.load(f)

found = []
missing = []
countries_geo = []
countries_json_list = [ c['properties']['name'] for c in geo_world['features']]

for country in df['country'].unique():   
    if country in countries_json_list:
        found.append(country)
    else:
        missing.append(country)

print(f'Countries found    : {len(found)}')
print(f'Countries not found: {len(missing)}')
# geo_world_ok = {'type': 'FeatureCollection', 'features': countries_geo}
print('\nBrakujące państwa lub źle zakodowane: ', missing)

Poniżej nazwy państw zostają poprawnie zakodowane.

In [ ]:
df.loc[df['country'] == 'Republic of Korea', 'country'] = 'Korea'
df.loc[df['country'] == 'Russian Federation', 'country'] = 'Russia'
df.loc[df['country'] == 'Bosnia and Herzegovina', 'country'] = 'Bosnia and Herz.'
df.loc[df['country'] == 'Czech Republic', 'country'] = 'Czech Rep.'
df.loc[df['country'] == 'Antigua and Barbuda', 'country'] = 'Antigua and Barb.'
df.loc[df['country'] == 'Cabo Verde', 'country'] = 'Cape Verde'
df.loc[df['country'] == 'Macau', 'country'] = 'Macao'
df.loc[df['country'] == 'Saint Kitts and Nevis', 'country'] = 'St. Kitts and Nevis'
df.loc[df['country'] == 'Saint Vincent and Grenadines', 'country'] = 'St. Vin. and Gren.'

### Selekcja danych

Na koniec procesu czyszczenia danych wyselekcjonujemy odpowiedni ich podzbiór. Na przestrzeni lat dane zawarte w zbiorze zmieniały się dynamicznie, ze względu na przeróżne zmiany społeczno gospodarcze zachodzące na całym świecie. Chcąc pokazać wiarygodne rezultaty badań, pojawia się konieczność pominięcia tych danych, które są niekompletne. Sprawdźmy ile mamy danych dla występujących w zbiorze lat.

In [ ]:
df['year'].value_counts()

Dla roku 2016 danych jest bardzo mało, więc dane dotyczące tego roku zostaną wykluczone.

In [ ]:
df = df[df['year'] <= 2015]

Gotowy zbiór zapisany zostaje w osobnym pliku.

In [ ]:
df.to_csv('data/suicides_cleaned.csv')

### Specyfika danych

Po wstępnym przetworzeniu można zobaczyć jak prezentuje się zbiór gotowy do przeprowadzania analiz. Przyjrzyjmy się najpierw poszczególnym kolumnom.

Jak widać poniżej, zbiór zawiera dane pochodzące ze 100 różnych krajów.

In [ ]:
df.country.unique().size

Liczba danych dla kobiet oraz mężczyzn jest jednakowa.

In [ ]:
df.sex.value_counts()

Dane dotyczą 6 różnych grup wiekowych.

In [ ]:
df.age.unique()

Warto zwrócić uwagę na to, że dystrybucja danych dla każdej z grup jest podobna.

In [ ]:
df.age.value_counts()

Statystyki dla omawianego zbioru są następujące:

In [ ]:
df.describe()

### Wybór odpowiedniej palety kolorów.

Aby zachować jednolitość prezentowanych wizualizacji, zdecydowano się na wybór odpowiedniej palety kolorów. Wybrano jedną z palet dostępnych jako [predefinowane](https://plotly.com/python/builtin-colorscales/) w bibliotece plotly. Wybrana paleta nosi nazwę **Inferno** i wybrana została, ponieważ jej kolory naturalnie kojarzą się z ilością ciepła, a więc również ogólnie z natężeniem wystąpień jakiegoś zjawiska. Paleta kolorów jest zaprezentowana poniżej. Można również wybrać z listy inną paletę, aby zobaczyć jakie inne palety oferuje biblioteka plotly.

In [ ]:
color_palette = px.colors.sequential.Inferno
color_palette_name = 'inferno'
def get_palette_discretes(start=2, end=len(color_palette)-1, step=3):
    return color_palette[start:end:step]

In [ ]:
colorscales = px.colors.named_colorscales()
df_iris = px.data.iris()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.P("Paleta kolorów:"),
    dcc.Dropdown(
        id='colorscale', 
        options=[{"value": x, "label": x} 
                 for x in colorscales],
        value=color_palette_name
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("colorscale", "value")])
def change_colorscale(scale):
    fig = px.scatter(
        df_iris, x="sepal_width", y="sepal_length", 
        color="sepal_length", color_continuous_scale=scale)
    return fig

app.run_server(mode='inline')

Poniżej pokazane zostało spektrum wybranej palety dla niewielkiego zbioru wartości.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=[list(range(20))],
    colorscale=color_palette,
    colorbar=dict(
        tick0=0,
        dtick=1
    )
))

fig.show()

## Analiza eksploracyjna

Zbiór został już wstępnie oczyszczony więc można zająć się teraz jego analizą. Odnalezione zostaną zależności między jego atrybutami i pokazane z użyciem interaktynych elementów.

### Liczba samobójstw na przestrzeni lat
Na samym początku zobaczmy ile osób na 100 tysięcy odbierało sobie życie w okresie od 1985 do 2015.

In [ ]:
df_gby_years = df.groupby(['year'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_years['suicides_per_100k'] = round((df_gby_years['suicides_no']/df_gby_years['population'])*100000, 2)
suicide_mean = round(df_gby_years['suicides_per_100k'].mean(), 2)

fig_overall = go.Figure(data=go.Scatter(x=df_gby_years['year'], y=df_gby_years['suicides_per_100k'], mode='lines+markers', marker_color=color_palette[4]))
fig_overall.add_hline(y=suicide_mean, line_dash="dash", line_color=color_palette[7], annotation_text=10*" " + "średnia = " + str(suicide_mean), annotation_position="top left")
fig_overall.update_layout(title="Samobójstwa na 100 tys. osób na przestrzni lat (globalnie)")
fig_overall.update_xaxes(title_text="rok")
fig_overall.update_yaxes(title_text="samobójstwa na 100 tys.")
fig_overall.show()

Nasze dane wskazują na to, że liczba samobójstw zmieniała się dynamicznie i **osiągnęła najwięszką wartość w roku 1995**, natomiast **średnia wartość wynosiła 13.12**.

### Samobójstwa a płeć

W tej sekcji przeanalizowany został nasz zestaw danych, sprawdzając, jak współczynniki samobójstw korelują z płcią. 

In [ ]:
df_gby_gender = df.groupby(['sex'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_gender['suicides_per_100k_in_group'] = round((df_gby_gender['suicides_no']/df_gby_gender['population'])*100000, 2)

fig_pie_gender = px.pie(df_gby_gender, values='suicides_per_100k_in_group', names='sex', title='Zależność od płci', 
                        color_discrete_sequence=get_palette_discretes(start=2,step=3), custom_data=['suicides_per_100k_in_group'])
fig_pie_gender.update_traces(textposition='auto', sort=False, 
                             hovertemplate="Płeć: <b>%{label}</b> <br> Liczba samobójstw na 100 tys. osób danej płci: <b>%{customdata[0]}</b>")
fig_pie_gender.show()

Z powyższego wykresu można dowiedzieć się, że wskaźnik samobójstw dla mężczyzn jest znacznie wyższy niż dla kobiet.

### Samobójstwa a wiek

Interesującą kwestią dotyczącą tego przykrego zjawiska jakim jest samobójstwo jest na pewno wiek osób, które najczęściej decydują się na taki krok. Zobaczmy jaki procent stanowią poszczególny grupy wiekowe.

In [ ]:
# agregacja danych i wyrkesy
df_gby_age = df.groupby(['age'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_age['suicides_per_100k_in_group'] = round((df_gby_age['suicides_no']/df_gby_age['population'])*100000, 2)
# 1 pie
fig_pie_age_no = px.pie(df_gby_age, names='age', values='suicides_no', color_discrete_sequence=get_palette_discretes(), custom_data=['suicides_no'])
fig_pie_age_no.update_traces(textposition='auto', textinfo='label', sort=False,
                             hovertemplate="Wiek: %{label}: <br><br><b>Procent: %{percent}</b> </br>Liczba samobójstw: %{customdata[0]}")
# 2 pie
fig_pie_age_100k = px.pie(df_gby_age, names='age', values='suicides_per_100k_in_group', color_discrete_sequence=get_palette_discretes(), custom_data=['suicides_per_100k_in_group']) 
fig_pie_age_100k.update_traces(textposition='auto', textinfo='label', sort=False,
                               hovertemplate="Wiek: %{label}: <br><br><b>Procent: %{percent}</b> </br>Liczba samobójstw na 100 tysięcy: %{customdata[0]}")

# dash layout
app.layout = html.Div([
    html.Br(),
    html.H2('Samobójstwa według wieku', style={'text-align':'center'}),
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Łącznie', value='tab-1'),
        dcc.Tab(label='Na 100 tysięcy osób danej grupy', value='tab-2'),
    ]),
    html.Div(id='tabs-content', children=[]),
    dcc.Graph(id='age_pie_chart', figure={})
])

@app.callback(Output(component_id='age_pie_chart', component_property='figure'),
              Input(component_id='tabs', component_property='value'))
def render_content(tab):
    
    if tab == 'tab-1':
        return fig_pie_age_no
    elif tab == 'tab-2':
        return fig_pie_age_100k
    
app.run_server(mode='inline')

**Na pierwszym z powyższych wykresów** widać wyraźnie, że największą grupą osób są te w wieku od **35 do 54 lat**. Mogłbo by się wydawać, że to właśnie osoby z tej grupy najczęściej popełniają samobójstwo.

Istotne jednak jest to, że grupy nie są sobie równe pod względem wielkości. Aby zobaczyć, w jakiej grupie wiekowej najwięcej osób odbiera sobie życie, należy pokazać te dane liczbowe dla 100 tysięcy osób z danej grupy tak jak pokazano **na wykresie drugim**. Należy zwrócić uwagę na to, że **grupa w tym wypadku to wszystkie osoby w danym przedziale wiekowym**. 

Analizując dane na wykresie przedstawiającym dane liczone na 100 tysięcy osób można zauważyć, że to osoby **w wieku powyżej 75 lat** najczęsciej decydują się na samobójstwo.

A jak powyższe liczby zmieniały się na przestrzeni lat? Czy trend był spadkowy? A może rosnący? Sprawdźmy najpierw zmiany dla łącznej liczby samobójstw w danych grupach na poniższej animacji. 

In [ ]:
df_gby_year_age = df.groupby(['year', 'age'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_year_age['suicides_per_100k_in_group'] = round((df_gby_year_age['suicides_no']/df_gby_year_age['population'])*100000, 2)
df_gby_year_age['temp'] = df_gby_year_age['age'].str.extract(r'(\d\d)')
df_gby_year_age.sort_values(by=['year', 'temp'], inplace=True)
colors = {'Ugly': 'red',
          'Bad': 'orange',
          'Good': 'lightgreen',
          'Great': 'darkgreen'}
fig_bar_animated = px.bar(df_gby_year_age, x="age", y="suicides_no", color='age', title="Liczba samobójstw na przestrzeni lat - animacja",  
                          color_discrete_sequence=get_palette_discretes(start=2, step=1), labels={'age':'Wiek', 'suicides_no':'Liczba samobójstw', 'year':'Rok'},
                          animation_frame="year", range_y=[0,110000]) 
fig_bar_animated.show()

Pokazując dane dla 100 tysięcy osób w danej grupie z użyciem wyrkesów liniowych możemy dokładnie zobaczyć zmiany zachodzące w czasie.

In [ ]:
year_min = df_gby_year_age['year'].min()
year_max = df_gby_year_age['year'].max()

app3 = JupyterDash(__name__)
app3.layout = html.Div([
    html.Br(),
    html.H4('Grupy wiekowe:'),
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} for x in df_gby_year_age['age'].unique()],
        value=df_gby_year_age['age'].unique(),
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart-year-age"),
    dcc.RangeSlider(
        id='range-slider',
        min=year_min, max=year_max, step=1,
        marks={int(year_min): str(year_min), int(year_max):str(year_max)},
        value=[year_min, year_max], pushable=2,
    ),
    html.Div(id='output-container-range-slider', style={'text-align':'center'}),
])

@app3.callback(
    [Output("line-chart-year-age", "figure"), Output('output-container-range-slider', 'children')], 
    [Input("checklist", "value"), Input('range-slider', 'value')])
def update_line_chart_year_age(groups, years):
    mask = (df_gby_year_age['age'].isin(groups)) & (df_gby_year_age['year'].between(*years))
#     mask = df_gby_year_age['year'].between(*years)
    fig_line_year_age = px.line(df_gby_year_age[mask], x="year", y="suicides_per_100k_in_group", 
                  color='age', color_discrete_sequence=get_palette_discretes(start=3, step=1), 
                  labels={'age':'Wiek', 'year':'rok', 'suicides_per_100k_in_group':'samobójstwa na 100 tys.'})
    
    years_selected = f'Lata: {years}'
    return fig_line_year_age, years_selected

app3.run_server(mode='inline', port=8070)

### Analiza geograficzna

Mając dane dotyczące poszczególnych krajów, można zobaczyć w których znich wskaźnik samobójstw był największy. Zobaczmy te dane dla 10 państw z największym wskaźnikiem.

In [ ]:
df_gby_country = df.groupby(['country'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_country['suicides_per_100k_in_group'] = round((df_gby_country['suicides_no']/df_gby_country['population'])*100000, 2)

fig_countries = px.bar(df_gby_country.nlargest(10, 'suicides_per_100k_in_group'), x='country', y='suicides_per_100k_in_group', color='suicides_per_100k_in_group',
                      color_discrete_sequence=color_palette, labels={'country':'kraj', 'suicides_per_100k_in_group':'liczba samobójstw na 100 tys.'})
fig_countries.show()

Powyższa wizualizacja wskazuje na to, że na **Litwie** liczba samobójstw jest największa ze wszystkich państw, na temat których posiadamy dane w zbiorze. Kolejnymi państwami z bardzo wysokim wskaźnikiem samobójstw jest **Rosja** oraz **Sri Lanka**. Na poniższym wykresie można wybrać państwa, dla których chcemy zobaczyć wskaźnik samobójstw na przestrzeni lat.

In [ ]:
df_gby_countries_years = df.groupby(['country','year'], as_index=False)[['suicides_no', 'population']].sum()
df_gby_countries_years['suicides_per_100k_in_group'] = (df_gby_countries_years['suicides_no']/df_gby_countries_years['population'])*100000


app2 = JupyterDash(__name__)
app2.layout = html.Div([
    html.Br(),
    dcc.Dropdown(
        id='countries-dropdown',
        options=[{'label':country, 'value':country} for country in df_gby_countries_years['country'].unique()],
        multi=True,
        value=['Poland', 'Russia', 'Germany']
    ),
    dcc.Graph(id="line-chart-countries-years"),
    html.Div(id='dd-output-container')
])

@app2.callback(
    Output("line-chart-countries-years", "figure"), 
    [Input("countries-dropdown", "value")])
def update_line_chart_countries_years(selected_countries):
    mask_countries_years = df_gby_countries_years['country'].isin(selected_countries)
    fig_countries_years = px.line(df_gby_countries_years[mask_countries_years], x="year", y="suicides_per_100k_in_group", 
                                  color='country', color_discrete_sequence=px.colors.qualitative.Prism, 
                                  labels={'country':'kraj', 'suicides_per_100k_in_group':'liczba samobójstw na 100 tys.', 'year':'rok'})
                            
    return fig_countries_years

app2.run_server(mode='inline', port=8060)

Aby ujrzeć dane dla wszystkich państw ze zbioru, zostaną one przedstawione na mapie. 

In [ ]:
json_map_path = 'data/custom.geo.json'
with open(json_map_path) as f:
    geo_world = json.load(f)

countries_geo = []

for country in geo_world['features']:
    
    country_name = country['properties']['name'] 
    
    if country_name in df_gby_country['country'].unique():
        geometry = country['geometry'] 
        
        countries_geo.append({
            'type': 'Feature',
            'geometry': geometry,
            'id':country_name
        })        
geo_world_ok = {'type': 'FeatureCollection', 'features': countries_geo}

In [ ]:
fig_countries_map = px.choropleth_mapbox(
    df_gby_country,
    geojson=geo_world_ok,
    locations='country',
    color='suicides_per_100k_in_group',
    color_continuous_scale=color_palette,
    hover_name='country',
    mapbox_style='open-street-map',
    zoom=1,
    center={'lat': 19, 'lon': 11},
    opacity=0.7
)

fig_countries_map.update_layout(
    margin={'r':0,'t':0,'l':0,'b':0},
    coloraxis_colorbar={'title':'Samobójstwa na 100 tys.'},
    title={'text': "Samobójstwa na świecie w latach 1985-2015"}   
)
fig_countries_map.show()

Powyższa mapa pokazuje, że omawiany zbiór posiada znaczące braki. Brakuje w nim informacji na temat wielu państw Afryki oraz Azji.

### Podsumowanie
W wyniku przeprowadzonej analizy wyciągnięto następujące wnioski.

1. Samobójstwa są częstsze wśród mężczyzn niż kobiet
2. Najwięcej samobójstw popełniono w 1995 roku. Od tamtego czasu globalny wskaźnij samobójstw systematycznie spadał.
3. Grupa wiekowa, w której występuje najwięcej samobójstw to osoby w wieku od 35 do 54 lat. 
4. Grupa wiekowa, w której samobójstwo popełnia najwięcej osób na 100 tys. osób w tej grupie to osoby w wieku powyżej 75 lat.
5. Najwięcej samobójstw na 100 tys. osób występiło na Litwie, w Rosji oraz na Sri Lance.